In [63]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager

from unidecode import unidecode
from bs4 import BeautifulSoup
import requests
import time

import pandas as pd
import numpy as np

# 1.0. Coletando os dados

## 1.1. Selenium -> Clicando e coletando o HTML

In [64]:
# determinando a url
url = "https://pt.wikipedia.org/wiki/Regi%C3%B5es_administrativas_da_cidade_do_Rio_de_Janeiro"

# escolhendo o navegador
option = Options()
option.headless = False
driver = webdriver.Firefox(options=option, executable_path=GeckoDriverManager().install() )

# abrindo a url
driver.get(url)

# sleed de 1 segundo
time.sleep(1)

# clicando no elemento ordenador da tabela
driver.find_element_by_xpath("//div[@class='mw-parser-output']//table//thead//tr//th").click()

# coletando o html da tabela
element = driver.find_element_by_xpath("//div[@class='mw-parser-output']//table[@class='sortable wikitable wm-collapse jquery-tablesorter']//tbody")
html_content = element.get_attribute('outerHTML')

time.sleep(2)

# saindo do navegador
driver.quit()



====== WebDriver manager ======
Current firefox version is 86.0
Get LATEST driver version for 86.0
Driver [/home/tcbm/.wdm/drivers/geckodriver/linux64/v0.29.1/geckodriver] found in cache


In [67]:
# Variável com o HTML
soup = BeautifulSoup(html_content, 'html.parser')

## 1.2. Separando - Subprefeitura / Regiao Adm 

In [69]:
teste_dict = {}

# lista de links das cidades
list_links = []

# lista de linhas
linhas = soup.find_all('tr')

# passando por cada linha
for key, value in enumerate(linhas):    
    list_infos = [] # lista de valores
    
    #lista de colunas da linha
    setor = value.find_all('td')
    
    # passando por coluna
    for key, value_2 in enumerate(setor):
        
        # buscando as cidades
        cidades = value_2.find_all('a')
        if key == 3: # se a coluna for a das cidades
            list_cidades = [] 
            
            # passando por cidade
            for value_3 in cidades:
                # salvando as cidades em uma lista
                list_cidades.append(value_3.get_text())
                list_links.append(value_3.get('href'))
                
        list_infos.append(value_2.get_text()) # salvando todas as informacoes em uma lista
        
    # passando as informacoes para cada cidade
    for value_4 in list_cidades:
        teste_dict[value_4] = list_infos

In [70]:
# Criando o DataFrame final
df_sub = pd.DataFrame(teste_dict).T.reset_index()
df_sub.drop([1, 3, 4], axis=1, inplace=True)
df_sub.columns = ['nome', 'subprefeitura', 'regiao_adm']

In [71]:
df_sub

,nome,subprefeitura,regiao_adm
0,Anil,Barra e Jacarepaguá,Jacarepaguá
1,Curicica,Barra e Jacarepaguá,Jacarepaguá
2,Freguesia (Jacarepaguá),Barra e Jacarepaguá,Jacarepaguá
3,Gardênia Azul,Barra e Jacarepaguá,Jacarepaguá
4,Jacarepaguá,Barra e Jacarepaguá,Jacarepaguá
...,...,...,...
158,Lagoa,Zona Sul,Lagoa
159,Leblon,Zona Sul,Lagoa
160,São Conrado,Zona Sul,Lagoa
161,Vidigal,Zona Sul,Lagoa


In [8]:
# Pequenas manipulações no DataFrame
df_sub['subprefeitura'] = df_sub['subprefeitura'].apply(lambda x: str(x).split('\n')[0])
df_sub['regiao_adm'] = df_sub['regiao_adm'].apply(lambda x: str(x).split('\n')[0])

In [20]:
df_sub['nome'] = df_sub['nome'].apply(lambda x: str(x).title())

## 1.3. Coletando - idh e area

In [9]:
# formatando em link completo
list_url = []
for value in list_links:
    list_url.append('http://pt.wikipedia.org' + value)

In [10]:
# fazendo a coleta
list_bairros = []

for url in list_url:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Achando a tabela com todas as informacoes
    table_infos = soup.find('table', class_='infobox')

    # Separando as informacoes em blocos
    # nome do bairro
    if table_infos != None:
        nome_bairro = table_infos.find('th').get_text()

        dict_infos = {}
        dict_infos['nome_bairro'] = nome_bairro
        
        # informacoes gerais
        for value in table_infos.find_all('tr'):
            index_info = value.find('td', attrs={'scope':'row'})
            if index_info:
                # valor da coluna
                value_info = value.find_all('td')[1]

                dict_infos[index_info.get_text()] = value_info.get_text()
        
        list_bairros.append(dict_infos)

# 2.0. Tratando os Dados

In [11]:
# df_bairros como um DataFrame
df_bairros = pd.DataFrame(list_bairros)

## 2.1. Limpando o titulo das colunas

In [12]:
columns = df_bairros.columns.to_list()

# limpeza das columns name

new_list_columns = []
for key, value in enumerate(columns):
    # primeira limpeza -> '\n'
    limp_1 = value.split('\n')[0]
    
    # segunda limpeza -> '• '
    if '•' in limp_1:
        limp_2 = limp_1.split('• ')[1]
    else:
        limp_2 = limp_1
    
    # terceira limpeza -> (ano)
    if '(' in limp_2:
        limp_3 = limp_2.split(' (')[0]
    else:
        limp_3 = limp_2
        
    if limp_3 not in new_list_columns:
        new_list_columns.append(limp_3)
    else:
        new_list_columns.append(limp_3 + f'{key}')
        
# Mudando o nome das colunas
df_bairros.columns = new_list_columns

In [13]:
# selecionando as colunas que irei usar
aux = df_bairros.loc[:, ['nome_bairro', 'Área', 'Área total', 'IDH', 'IDH14']].copy()

## 2.2. Juntando valores que estão separados

In [14]:
# nome_bairro
# Área + Área total -> area
# IDH + IDH14 -> idh

In [15]:
# juntando as colunas area
serie_area = aux['Área total']
aux.loc[:, 'area'] = aux['Área'].fillna(serie_area).copy()

# juntando as colunas idh
serie_idh = aux['IDH14']
aux.loc[:, 'idh'] = aux['IDH'].fillna(serie_idh).copy()

In [16]:
# df_scrap final para fazer os devidos tratamentos
df_scrap = aux.loc[:, ['nome_bairro', 'area', 'idh']].copy()

## 2.3. Tratamentos

### 2.3.1. Area

In [17]:
# tratando area

# fazendo os tratamento necessários
df_scrap['area'] = df_scrap['area'].apply(lambda x: str(x).replace('.', '').replace(' ', '').split('(')[0].replace(',', '.').replace('\xa0', ''))

# separando em duas colunas provisórias valores em hectares e valores em km
df_scrap['area_h'] = df_scrap['area'].apply(lambda x: x.split('h')[0] if 'h' in str(x) else np.nan).astype('float64')
df_scrap['area_k'] = df_scrap['area'].apply(lambda x: x.split('k')[0].split('[')[0] if 'k' in str(x) else np.nan).astype('float64') * 100

# resultado final
df_scrap['area_final'] = df_scrap['area_h'].fillna(df_scrap['area_k'])

### 2.3.2. IDH

In [18]:
# tratando idh

# fazendo os tratamentos necessários
df_scrap['idh_final'] = df_scrap['idh'].apply(lambda x: str(x).split('[')[0].split(' ')[0].split('(')[0].split('\n')[0].replace(',', '.')).astype('float64')

### 2.3.3. Nome Bairro

In [19]:
# tratando nome_bairro

# fazendo os tratamentos necessários
df_scrap['nome_bairro_final'] = df_scrap['nome_bairro'].apply(lambda x: str(x).split('(')[0].replace('\n', '').split('"')[0].rstrip().title())

### 2.3.4. DF Final

In [21]:
# df_result como DataFrame final
df_result = df_scrap.loc[:, ['nome_bairro_final', 'area_final', 'idh_final']]
df_result.columns = ['nome', 'area', 'idh']

## 2.4. Ultimas Manipulações

In [23]:
# Excluindo ascentos
df_result['nome'] = df_result['nome'].apply(lambda x: unidecode(x)).copy()
df_sub['nome'] = df_sub['nome'].apply(lambda x: unidecode(x)).copy()

In [24]:
# Algumas manipulacoes manuais para o merge
df_result.loc[df_result['nome'] == 'Largo Do Anil', 'nome'] = 'Anil'
df_result.loc[df_result['nome'] == 'Freguesia De Jacarepagua', 'nome'] = 'Freguesia (Jacarepagua)'
df_result.loc[df_result['nome'] == 'Freguesia', 'nome'] = 'Freguesia (Ilha Do Governador)'
df_sub.loc[df_sub['nome'] == 'Freguesia', 'nome'] = 'Freguesia (Ilha Do Governador)'
df_result.loc[df_result['nome'] == 'Largo Do Visconde De Asseca', 'nome'] = 'Praca Seca'
df_result.loc[df_result['nome'] == 'Largo Do Tanque', 'nome'] = 'Tanque'
df_sub.loc[df_sub['nome'] == 'Cavalcante', 'nome'] = 'Cavalcanti'
df_result.loc[df_result['nome'] == ' Parque Anchieta', 'nome'] = 'Parque Anchieta'
df_result.loc[df_result['nome'] == 'Senador Augusto Vasconcelos', 'nome'] = 'Senador Vasconcelos'
df_result.loc[df_result['nome'] == 'Aerodromo Campo Dos Afonsos Base Aerea Dos Afonsos Campo Dos Afonsos', 'nome'] = 'Campo Dos Afonsos'

# 3.0. DataFrames Finais

In [27]:
df_result.to_csv('area_idh_data.csv', encoding='utf-8', index=False)

In [28]:
df_sub.to_csv('sub_adm_data.csv', encoding='utf-8', index=False)